<a href="https://colab.research.google.com/github/del-luna/aiffel/blob/main/going_deeper_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds

In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    'cats_vs_dogs',
    split = ('train[:80%]', 'train[80%:]'),
    with_info = True,
    as_supervised = True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incompleteQ2FSVB/cats_vs_dogs-train.tfrecord


  0%|          | 0/23262 [00:00<?, ? examples/s]

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.


In [3]:
def format_image(image, label):
    image = tf.image.resize(image, (SIZE, SIZE))/255.0
    return image, label

num_examples = ds_info.splits['train'].num_examples

BATCH_SIZE = 64
SIZE = 224

train_batches = ds_train.cache().shuffle(num_examples//4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = ds_test.map(format_image).batch(BATCH_SIZE).prefetch(1)

In [11]:
class BasicBlock2(tf.keras.layers.Layer):

    def __init__(self, filter_num, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=filter_num,
                                            kernel_size=(3,3),
                                            strides=stride,
                                            padding='same')
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filters=filter_num,
                                            kernel_size=(3,3),
                                            strides=1,
                                            padding='same')
        self.bn2 = tf.keras.layers.BatchNormalization()
        if stride != 1:
            self.shortcut = tf.keras.Sequential()
            self.shortcut.add(tf.keras.layers.Conv2D(filters=filter_num,
                                                     kernel_size=(1,1),
                                                     strides=stride))
            self.shortcut.add(tf.keras.layers.BatchNormalization())
        else:
            self.shortcut = lambda x: x

    def call(self, inputs, training=None, **kwargs):
        residual = self.shortcut(inputs)

        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x)
        x = self.conv2(x)
        x = self.bn2(x, training=training)

        output = tf.nn.relu(x)

        return output

class BottleNeck2(tf.keras.layers.Layer):
    def __init__(self, filter_num, stride=1):
        super(BottleNeck, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=filter_num,
                                            kernel_size=(1, 1),
                                            strides=1,
                                            padding='same')
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filters=filter_num,
                                            kernel_size=(3, 3),
                                            strides=stride,
                                            padding='same')
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.conv3 = tf.keras.layers.Conv2D(filters=filter_num * 4,
                                            kernel_size=(1, 1),
                                            strides=1,
                                            padding='same')
        self.bn3 = tf.keras.layers.BatchNormalization()

        self.downsample = tf.keras.Sequential()
        self.downsample.add(tf.keras.layers.Conv2D(filters=filter_num * 4,
                                                   kernel_size=(1, 1),
                                                   strides=stride))
        self.downsample.add(tf.keras.layers.BatchNormalization())

    def call(self, inputs, training=None, **kwargs):
        residual = self.downsample(inputs)

        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x)
        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = tf.nn.relu(x)
        x = self.conv3(x)
        x = self.bn3(x, training=training)

        output = tf.nn.relu(x)

        return output

def make_basic_block_layer2(filter_num, blocks, stride=1):
    res_block = tf.keras.Sequential()
    res_block.add(BasicBlock(filter_num, stride=stride))

    for _ in range(1, blocks):
        res_block.add(BasicBlock(filter_num, stride=1))

    return res_block


def make_bottleneck_layer2(filter_num, blocks, stride=1):
    res_block = tf.keras.Sequential()
    res_block.add(BottleNeck(filter_num, stride=stride))

    for _ in range(1, blocks):
        res_block.add(BottleNeck(filter_num, stride=1))

    return res_block

In [12]:
class ResNetTypeIII(tf.keras.Model):
    def __init__(self, layer_params):
        super(ResNetTypeI, self).__init__()

        self.conv1 = tf.keras.layers.Conv2D(filters=64,
                                            kernel_size=(7, 7),
                                            strides=2,
                                            padding="same")
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=(3, 3),
                                               strides=2,
                                               padding="same")

        self.layer1 = make_basic_block_layer2(filter_num=64,
                                             blocks=layer_params[0])
        self.layer2 = make_basic_block_layer2(filter_num=128,
                                             blocks=layer_params[1],
                                             stride=2)
        self.layer3 = make_basic_block_layer2(filter_num=256,
                                             blocks=layer_params[2],
                                             stride=2)
        self.layer4 = make_basic_block_layer2(filter_num=512,
                                             blocks=layer_params[3],
                                             stride=2)

        self.avgpool = tf.keras.layers.GlobalAveragePooling2D()
        self.fc = tf.keras.layers.Dense(units=2, activation=tf.keras.activations.softmax)

    def call(self, inputs, training=None, mask=None):
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x)
        x = self.pool1(x)
        x = self.layer1(x, training=training)
        x = self.layer2(x, training=training)
        x = self.layer3(x, training=training)
        x = self.layer4(x, training=training)
        x = self.avgpool(x)
        output = self.fc(x)

        return output


class ResNetTypeIV(tf.keras.Model):
    def __init__(self, layer_params):
        super(ResNetTypeII, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=64,
                                            kernel_size=(7, 7),
                                            strides=2,
                                            padding="same")
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=(3, 3),
                                               strides=2,
                                               padding="same")

        self.layer1 = make_bottleneck_layer2(filter_num=64,
                                            blocks=layer_params[0])
        self.layer2 = make_bottleneck_layer2(filter_num=128,
                                            blocks=layer_params[1],
                                            stride=2)
        self.layer3 = make_bottleneck_layer2(filter_num=256,
                                            blocks=layer_params[2],
                                            stride=2)
        self.layer4 = make_bottleneck_layer2(filter_num=512,
                                            blocks=layer_params[3],
                                            stride=2)

        self.avgpool = tf.keras.layers.GlobalAveragePooling2D()
        self.fc = tf.keras.layers.Dense(units=2, activation=tf.keras.activations.softmax)

    def call(self, inputs, training=None, mask=None):
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x)
        x = self.pool1(x)
        x = self.layer1(x, training=training)
        x = self.layer2(x, training=training)
        x = self.layer3(x, training=training)
        x = self.layer4(x, training=training)
        x = self.avgpool(x)
        output = self.fc(x)

        return output


def resnet_18_2():
    return ResNetTypeIII(layer_params=[2, 2, 2, 2])


def resnet_34_2():
    return ResNetTypeIII(layer_params=[3, 4, 6, 3])


def resnet_50_2():
    return ResNetTypeIV(layer_params=[3, 4, 6, 3])


def resnet_101_2():
    return ResNetTypeIV(layer_params=[3, 4, 23, 3])


def resnet_152_2():
    return ResNetTypeIV(layer_params=[3, 8, 36, 3])

In [4]:
class BasicBlock(tf.keras.layers.Layer):

    def __init__(self, filter_num, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=filter_num,
                                            kernel_size=(3,3),
                                            strides=stride,
                                            padding='same')
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filters=filter_num,
                                            kernel_size=(3,3),
                                            strides=1,
                                            padding='same')
        self.bn2 = tf.keras.layers.BatchNormalization()
        if stride != 1:
            self.shortcut = tf.keras.Sequential()
            self.shortcut.add(tf.keras.layers.Conv2D(filters=filter_num,
                                                     kernel_size=(1,1),
                                                     strides=stride))
            self.shortcut.add(tf.keras.layers.BatchNormalization())
        else:
            self.shortcut = lambda x: x

    def call(self, inputs, training=None, **kwargs):
        residual = self.shortcut(inputs)

        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x)
        x = self.conv2(x)
        x = self.bn2(x, training=training)

        output = tf.nn.relu(tf.keras.layers.add([residual, x]))

        return output

class BottleNeck(tf.keras.layers.Layer):
    def __init__(self, filter_num, stride=1):
        super(BottleNeck, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=filter_num,
                                            kernel_size=(1, 1),
                                            strides=1,
                                            padding='same')
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filters=filter_num,
                                            kernel_size=(3, 3),
                                            strides=stride,
                                            padding='same')
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.conv3 = tf.keras.layers.Conv2D(filters=filter_num * 4,
                                            kernel_size=(1, 1),
                                            strides=1,
                                            padding='same')
        self.bn3 = tf.keras.layers.BatchNormalization()

        self.downsample = tf.keras.Sequential()
        self.downsample.add(tf.keras.layers.Conv2D(filters=filter_num * 4,
                                                   kernel_size=(1, 1),
                                                   strides=stride))
        self.downsample.add(tf.keras.layers.BatchNormalization())

    def call(self, inputs, training=None, **kwargs):
        residual = self.downsample(inputs)

        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x)
        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = tf.nn.relu(x)
        x = self.conv3(x)
        x = self.bn3(x, training=training)

        output = tf.nn.relu(tf.keras.layers.add([residual, x]))

        return output

def make_basic_block_layer(filter_num, blocks, stride=1):
    res_block = tf.keras.Sequential()
    res_block.add(BasicBlock(filter_num, stride=stride))

    for _ in range(1, blocks):
        res_block.add(BasicBlock(filter_num, stride=1))

    return res_block


def make_bottleneck_layer(filter_num, blocks, stride=1):
    res_block = tf.keras.Sequential()
    res_block.add(BottleNeck(filter_num, stride=stride))

    for _ in range(1, blocks):
        res_block.add(BottleNeck(filter_num, stride=1))

    return res_block

In [5]:
class ResNetTypeI(tf.keras.Model):
    def __init__(self, layer_params):
        super(ResNetTypeI, self).__init__()

        self.conv1 = tf.keras.layers.Conv2D(filters=64,
                                            kernel_size=(7, 7),
                                            strides=2,
                                            padding="same")
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=(3, 3),
                                               strides=2,
                                               padding="same")

        self.layer1 = make_basic_block_layer(filter_num=64,
                                             blocks=layer_params[0])
        self.layer2 = make_basic_block_layer(filter_num=128,
                                             blocks=layer_params[1],
                                             stride=2)
        self.layer3 = make_basic_block_layer(filter_num=256,
                                             blocks=layer_params[2],
                                             stride=2)
        self.layer4 = make_basic_block_layer(filter_num=512,
                                             blocks=layer_params[3],
                                             stride=2)

        self.avgpool = tf.keras.layers.GlobalAveragePooling2D()
        self.fc = tf.keras.layers.Dense(units=2, activation=tf.keras.activations.softmax)

    def call(self, inputs, training=None, mask=None):
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x)
        x = self.pool1(x)
        x = self.layer1(x, training=training)
        x = self.layer2(x, training=training)
        x = self.layer3(x, training=training)
        x = self.layer4(x, training=training)
        x = self.avgpool(x)
        output = self.fc(x)

        return output


class ResNetTypeII(tf.keras.Model):
    def __init__(self, layer_params):
        super(ResNetTypeII, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=64,
                                            kernel_size=(7, 7),
                                            strides=2,
                                            padding="same")
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=(3, 3),
                                               strides=2,
                                               padding="same")

        self.layer1 = make_bottleneck_layer(filter_num=64,
                                            blocks=layer_params[0])
        self.layer2 = make_bottleneck_layer(filter_num=128,
                                            blocks=layer_params[1],
                                            stride=2)
        self.layer3 = make_bottleneck_layer(filter_num=256,
                                            blocks=layer_params[2],
                                            stride=2)
        self.layer4 = make_bottleneck_layer(filter_num=512,
                                            blocks=layer_params[3],
                                            stride=2)

        self.avgpool = tf.keras.layers.GlobalAveragePooling2D()
        self.fc = tf.keras.layers.Dense(units=2, activation=tf.keras.activations.softmax)

    def call(self, inputs, training=None, mask=None):
        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = tf.nn.relu(x)
        x = self.pool1(x)
        x = self.layer1(x, training=training)
        x = self.layer2(x, training=training)
        x = self.layer3(x, training=training)
        x = self.layer4(x, training=training)
        x = self.avgpool(x)
        output = self.fc(x)

        return output


def resnet_18():
    return ResNetTypeI(layer_params=[2, 2, 2, 2])


def resnet_34():
    return ResNetTypeI(layer_params=[3, 4, 6, 3])


def resnet_50():
    return ResNetTypeII(layer_params=[3, 4, 6, 3])


def resnet_101():
    return ResNetTypeII(layer_params=[3, 4, 23, 3])


def resnet_152():
    return ResNetTypeII(layer_params=[3, 8, 36, 3])

In [6]:
def get_model(type='resnet34'):
    if type == 'resenet50':
        model = resnet_50()
    else:
        model = resnet_34()
    model.build(input_shape=(None, 224, 224, 3))
    model.summary()
    return model

gpus = tf.config.experimental.list_physical_devices('GPU')
model = get_model()

Model: "res_net_type_i"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  9472      
                                                                 
 batch_normalization (BatchN  multiple                 256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  multiple                 0         
 )                                                               
                                                                 
 sequential (Sequential)     (None, 56, 56, 64)        223104    
                                                                 
 sequential_1 (Sequential)   (None, 28, 28, 128)       1119872   
                                                                 
 sequential_3 (Sequential)   (None, 14, 14, 256)    

In [7]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adadelta()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

valid_loss = tf.keras.metrics.Mean(name='valid_loss')
valid_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='valid_accuracy')

In [9]:
import math

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(y_true=labels, y_pred=predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(grads_and_vars=zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def valid_step(images, labels):
    predictions = model(images, training=False)
    v_loss = loss_object(labels, predictions)

    valid_loss(v_loss)
    valid_accuracy(labels, predictions)

# start training
for epoch in range(30):
    train_loss.reset_states()
    train_accuracy.reset_states()
    valid_loss.reset_states()
    valid_accuracy.reset_states()
    step = 0
    for images, labels in train_batches:
        step += 1
        train_step(images, labels)
        print("Epoch: {}/{}, step: {}/{}, loss: {:.5f}, accuracy: {:.5f}".format(epoch + 1,
                                                                                    30,
                                                                                    step,
                                                                                    math.ceil(num_examples / 64),
                                                                                    train_loss.result(),
                                                                                    train_accuracy.result()))

    for valid_images, valid_labels in validation_batches:
        valid_step(valid_images, valid_labels)

    print("Epoch: {}/{}, train loss: {:.5f}, train accuracy: {:.5f}, "
            "valid loss: {:.5f}, valid accuracy: {:.5f}".format(epoch + 1,
                                                                30,
                                                                train_loss.result(),
                                                                train_accuracy.result(),
                                                                valid_loss.result(),
                                                                valid_accuracy.result()))

Epoch: 21/30, step: 99/364, loss: 0.33342, accuracy: 0.86253
Epoch: 21/30, step: 100/364, loss: 0.33380, accuracy: 0.86203
Epoch: 21/30, step: 101/364, loss: 0.33355, accuracy: 0.86200
Epoch: 21/30, step: 102/364, loss: 0.33278, accuracy: 0.86259
Epoch: 21/30, step: 103/364, loss: 0.33211, accuracy: 0.86317
Epoch: 21/30, step: 104/364, loss: 0.33142, accuracy: 0.86343
Epoch: 21/30, step: 105/364, loss: 0.33224, accuracy: 0.86250
Epoch: 21/30, step: 106/364, loss: 0.33200, accuracy: 0.86232
Epoch: 21/30, step: 107/364, loss: 0.33175, accuracy: 0.86244
Epoch: 21/30, step: 108/364, loss: 0.33143, accuracy: 0.86299
Epoch: 21/30, step: 109/364, loss: 0.33262, accuracy: 0.86282
Epoch: 21/30, step: 110/364, loss: 0.33270, accuracy: 0.86264
Epoch: 21/30, step: 111/364, loss: 0.33228, accuracy: 0.86318
Epoch: 21/30, step: 112/364, loss: 0.33188, accuracy: 0.86356
Epoch: 21/30, step: 113/364, loss: 0.33160, accuracy: 0.86352
Epoch: 21/30, step: 114/364, loss: 0.33148, accuracy: 0.86376
Epoch: 21

In [10]:
gpus = tf.config.experimental.list_physical_devices('GPU')
model = get_model('resnet50')

import math

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(y_true=labels, y_pred=predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(grads_and_vars=zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def valid_step(images, labels):
    predictions = model(images, training=False)
    v_loss = loss_object(labels, predictions)

    valid_loss(v_loss)
    valid_accuracy(labels, predictions)

# start training
for epoch in range(30):
    train_loss.reset_states()
    train_accuracy.reset_states()
    valid_loss.reset_states()
    valid_accuracy.reset_states()
    step = 0
    for images, labels in train_batches:
        step += 1
        train_step(images, labels)
        print("Epoch: {}/{}, step: {}/{}, loss: {:.5f}, accuracy: {:.5f}".format(epoch + 1,
                                                                                    30,
                                                                                    step,
                                                                                    math.ceil(num_examples / 64),
                                                                                    train_loss.result(),
                                                                                    train_accuracy.result()))

    for valid_images, valid_labels in validation_batches:
        valid_step(valid_images, valid_labels)

    print("Epoch: {}/{}, train loss: {:.5f}, train accuracy: {:.5f}, "
            "valid loss: {:.5f}, valid accuracy: {:.5f}".format(epoch + 1,
                                                                30,
                                                                train_loss.result(),
                                                                train_accuracy.result(),
                                                                valid_loss.result(),
                                                                valid_accuracy.result()))

Streaming output truncated to the last 5000 lines.
Epoch: 13/30, step: 257/364, loss: 0.46409, accuracy: 0.78861
Epoch: 13/30, step: 258/364, loss: 0.46421, accuracy: 0.78858
Epoch: 13/30, step: 259/364, loss: 0.46444, accuracy: 0.78843
Epoch: 13/30, step: 260/364, loss: 0.46428, accuracy: 0.78858
Epoch: 13/30, step: 261/364, loss: 0.46414, accuracy: 0.78885
Epoch: 13/30, step: 262/364, loss: 0.46428, accuracy: 0.78870
Epoch: 13/30, step: 263/364, loss: 0.46424, accuracy: 0.78874
Epoch: 13/30, step: 264/364, loss: 0.46431, accuracy: 0.78847
Epoch: 13/30, step: 265/364, loss: 0.46441, accuracy: 0.78838
Epoch: 13/30, step: 266/364, loss: 0.46468, accuracy: 0.78812
Epoch: 13/30, step: 267/364, loss: 0.46479, accuracy: 0.78798
Epoch: 13/30, step: 268/364, loss: 0.46461, accuracy: 0.78807
Epoch: 13/30, step: 269/364, loss: 0.46476, accuracy: 0.78810
Epoch: 13/30, step: 270/364, loss: 0.46480, accuracy: 0.78819
Epoch: 13/30, step: 271/364, loss: 0.46461, accuracy: 0.78857
Epoch: 13/30, step:

In [13]:
def get_model(type='resnet34_2'):
    if type == 'resenet50_2':
        model = resnet_50_2()
    else:
        model = resnet_34()
    model.build(input_shape=(None, 224, 224, 3))
    model.summary()
    return model

gpus = tf.config.experimental.list_physical_devices('GPU')
model = get_model()

import math

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(y_true=labels, y_pred=predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(grads_and_vars=zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def valid_step(images, labels):
    predictions = model(images, training=False)
    v_loss = loss_object(labels, predictions)

    valid_loss(v_loss)
    valid_accuracy(labels, predictions)

# start training
for epoch in range(30):
    train_loss.reset_states()
    train_accuracy.reset_states()
    valid_loss.reset_states()
    valid_accuracy.reset_states()
    step = 0
    for images, labels in train_batches:
        step += 1
        train_step(images, labels)
        print("Epoch: {}/{}, step: {}/{}, loss: {:.5f}, accuracy: {:.5f}".format(epoch + 1,
                                                                                    30,
                                                                                    step,
                                                                                    math.ceil(num_examples / 64),
                                                                                    train_loss.result(),
                                                                                    train_accuracy.result()))

    for valid_images, valid_labels in validation_batches:
        valid_step(valid_images, valid_labels)

    print("Epoch: {}/{}, train loss: {:.5f}, train accuracy: {:.5f}, "
            "valid loss: {:.5f}, valid accuracy: {:.5f}".format(epoch + 1,
                                                                30,
                                                                train_loss.result(),
                                                                train_accuracy.result(),
                                                                valid_loss.result(),
                                                                valid_accuracy.result()))

Streaming output truncated to the last 5000 lines.
Epoch: 13/30, step: 257/364, loss: 0.47082, accuracy: 0.78052
Epoch: 13/30, step: 258/364, loss: 0.47073, accuracy: 0.78064
Epoch: 13/30, step: 259/364, loss: 0.47076, accuracy: 0.78083
Epoch: 13/30, step: 260/364, loss: 0.47075, accuracy: 0.78077
Epoch: 13/30, step: 261/364, loss: 0.47058, accuracy: 0.78107
Epoch: 13/30, step: 262/364, loss: 0.47037, accuracy: 0.78125
Epoch: 13/30, step: 263/364, loss: 0.47040, accuracy: 0.78131
Epoch: 13/30, step: 264/364, loss: 0.47024, accuracy: 0.78137
Epoch: 13/30, step: 265/364, loss: 0.47001, accuracy: 0.78166
Epoch: 13/30, step: 266/364, loss: 0.46973, accuracy: 0.78201
Epoch: 13/30, step: 267/364, loss: 0.46964, accuracy: 0.78201
Epoch: 13/30, step: 268/364, loss: 0.46982, accuracy: 0.78166
Epoch: 13/30, step: 269/364, loss: 0.47017, accuracy: 0.78142
Epoch: 13/30, step: 270/364, loss: 0.47009, accuracy: 0.78154
Epoch: 13/30, step: 271/364, loss: 0.47010, accuracy: 0.78160
Epoch: 13/30, step:

In [14]:
def get_model(type='resnet34_2'):
    if type == 'resenet50_2':
        model = resnet_50_2()
    else:
        model = resnet_34()
    model.build(input_shape=(None, 224, 224, 3))
    model.summary()
    return model

gpus = tf.config.experimental.list_physical_devices('GPU')
model = get_model('resnet_50_2')

import math

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(y_true=labels, y_pred=predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(grads_and_vars=zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def valid_step(images, labels):
    predictions = model(images, training=False)
    v_loss = loss_object(labels, predictions)

    valid_loss(v_loss)
    valid_accuracy(labels, predictions)

# start training
for epoch in range(30):
    train_loss.reset_states()
    train_accuracy.reset_states()
    valid_loss.reset_states()
    valid_accuracy.reset_states()
    step = 0
    for images, labels in train_batches:
        step += 1
        train_step(images, labels)
        print("Epoch: {}/{}, step: {}/{}, loss: {:.5f}, accuracy: {:.5f}".format(epoch + 1,
                                                                                    30,
                                                                                    step,
                                                                                    math.ceil(num_examples / 64),
                                                                                    train_loss.result(),
                                                                                    train_accuracy.result()))

    for valid_images, valid_labels in validation_batches:
        valid_step(valid_images, valid_labels)

    print("Epoch: {}/{}, train loss: {:.5f}, train accuracy: {:.5f}, "
            "valid loss: {:.5f}, valid accuracy: {:.5f}".format(epoch + 1,
                                                                30,
                                                                train_loss.result(),
                                                                train_accuracy.result(),
                                                                valid_loss.result(),
                                                                valid_accuracy.result()))

Streaming output truncated to the last 5000 lines.
Epoch: 13/30, step: 257/364, loss: 0.46919, accuracy: 0.78247
Epoch: 13/30, step: 258/364, loss: 0.46898, accuracy: 0.78270
Epoch: 13/30, step: 259/364, loss: 0.46880, accuracy: 0.78282
Epoch: 13/30, step: 260/364, loss: 0.46897, accuracy: 0.78263
Epoch: 13/30, step: 261/364, loss: 0.46885, accuracy: 0.78281
Epoch: 13/30, step: 262/364, loss: 0.46856, accuracy: 0.78298
Epoch: 13/30, step: 263/364, loss: 0.46873, accuracy: 0.78291
Epoch: 13/30, step: 264/364, loss: 0.46874, accuracy: 0.78297
Epoch: 13/30, step: 265/364, loss: 0.46862, accuracy: 0.78314
Epoch: 13/30, step: 266/364, loss: 0.46841, accuracy: 0.78336
Epoch: 13/30, step: 267/364, loss: 0.46831, accuracy: 0.78365
Epoch: 13/30, step: 268/364, loss: 0.46821, accuracy: 0.78382
Epoch: 13/30, step: 269/364, loss: 0.46818, accuracy: 0.78375
Epoch: 13/30, step: 270/364, loss: 0.46801, accuracy: 0.78403
Epoch: 13/30, step: 271/364, loss: 0.46801, accuracy: 0.78408
Epoch: 13/30, step:



*   Plain Network
    - Plain_34
        - train: 0.94, valid: 0.68
    - Plain_50
        - train: 0.95, valid: 0.66
*   ResNet
    - ResNet_34
        - train: 0.93, valid: 0.69
    - ResNet_50
        - train: 0.95, valid: 0.68
